In [2]:
import sys
import os
import importlib
import json
import pickle
import uuid
import copy
import tqdm
import requests
from urllib.parse import urlparse, unquote
from pathlib import Path

curr_path = Path.cwd()
while curr_path.name != 'tasks' and curr_path != curr_path.parent:
    curr_path = curr_path.parent

if curr_path.name == 'tasks':
    sys.path.append(str(curr_path))
else:
    raise FileNotFoundError("Could not find 'tasks' folder in parent hierarchy.")

import numpy as np
import pandas as pd

from pack import helper as h
from pack.wiki import wikipedia_connect as wc

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON

In [4]:
import wikipediaapi
import re
import requests
from urllib.parse import urlparse, unquote

In [4]:
# core_urls = [
#     'https://en.wikipedia.org/wiki/List_of_Chinese_contemporary_artists',
#     'https://en.wikipedia.org/wiki/List_of_Chinese_women_artists',
#     'https://en.wikipedia.org/wiki/List_of_Chinese_astronauts',
#     'https://en.wikipedia.org/wiki/List_of_Chinese_film_directors',
#     'https://en.wikipedia.org/wiki/List_of_Chinese_mathematicians',
#     'https://en.wikipedia.org/wiki/List_of_Chinese_musicians',
#     'https://en.wikipedia.org/wiki/List_of_Chinese_photographers'
# ]

In [5]:
core_urls = [
    'https://en.wikipedia.org/wiki/Party_and_state_leaders',
    'https://en.wikipedia.org/wiki/List_of_Chinese_by_net_worth'
]

In [6]:
def extract_sections(sections, page_title, base_url):
    sections_data = []

    def add_sections(sections):
        section_list = []
        for section in sections:
            section_data = {
                'section': section.title,
                'text': section.text,
                'url': f"{base_url}/wiki/{page_title.replace(' ', '_')}#{section.title.replace(' ', '_')}",
                'sub': add_sections(section.sections)
            }
            section_list.append(section_data)
        return section_list

    sections_data = add_sections(sections)
    return sections_data

def extract_personal_data(page):
    base_url = 'https://en.wikipedia.org'
    data = {
        'lang': 'en',
        'title': page.title,
        'summary': page.summary,
        'url': page.fullurl,
        'sections': extract_sections(page.sections, page.title, base_url),
        'languages': list(page.langlinks.keys()),
        'categories': list(page.categories.keys()),
        'links': list(page.links.keys()),
        'backlinks': [f"{base_url}/wiki/{title.replace(' ', '_')}" for title in page.backlinks.keys()],
        # 'images': extract_images(page.fullurl)
    }
    return data

In [6]:
# person_name = 'Freddie Mercury'
# wiki_wiki = wikipediaapi.Wikipedia(
#     user_agent='DataConnect (dataConnect@example.com)',
#     language='en',
#     extract_format=wikipediaapi.ExtractFormat.WIKI
# )
# page = wiki_wiki.page(person_name)

# if not page.exists():
#     raise ValueError('page does not exist')

# result_page = extract_personal_data(page)

In [7]:
# result_page

# New trial

In [7]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

def extract_people_links(list_url):
    base_url = "https://en.wikipedia.org"
    response = requests.get(list_url)
    soup = BeautifulSoup(response.content, 'html.parser')

    links = set()
    for li in soup.select('li a[href^="/wiki/"]'):
        href = li.get('href')
        # Filter out non-article links (skip files, categories, help pages, etc.)
        if not any(href.startswith(f"/wiki/{ns}:") for ns in ['Help', 'File', 'Category', 'Wikipedia', 'Template', 'Portal', 'Talk']):
            full_url = urljoin(base_url, href)
            links.add(full_url)

    return list(links)

## Access query.wikidata.org

```
SELECT ?item WHERE {
  ?item rdfs:label "Chang Ch'ung-ho"@en .
  ?article schema:about ?item ;
           schema:isPartOf <https://en.wikipedia.org/> .
}
LIMIT 1
```

#https://en.wikipedia.org/wiki/Special:EntityPage/Q15869

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

trial

In [10]:
# # Setup Edge WebDriver
# # service = Service(EdgeChromiumDriverManager().install())
# service = Service("./msedgedriver.exe")  # "./" means same folder as notebook
# options = webdriver.EdgeOptions()
# # options.add_argument("--headless")  # Optional: Uncomment for headless mode
# driver = webdriver.Edge(service=service, options=options)

# # Go to Wikidata Query Service
# driver.get("https://query.wikidata.org/")

# # Prepare your SPARQL query
# sparql_query = '''
# SELECT ?item WHERE {
#   ?item rdfs:label "Chang Ch'ung-ho"@en .
#   ?article schema:about ?item ;
#            schema:isPartOf <https://en.wikipedia.org/> .
# }
# LIMIT 1
# '''

# try:
#     # Wait for CodeMirror to load
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CLASS_NAME, "CodeMirror"))
#     )
#     time.sleep(2)

#     # Click on CodeMirror to focus
#     cm_editor = driver.find_element(By.CLASS_NAME, "CodeMirror")
#     cm_editor.click()
#     time.sleep(1)

#     # Locate the hidden textarea inside CodeMirror
#     textarea = cm_editor.find_element(By.CSS_SELECTOR, "textarea")
#     textarea.send_keys(Keys.CONTROL + "a")
#     textarea.send_keys(Keys.DELETE)
#     textarea.send_keys(sparql_query)
#     time.sleep(1)

#     # Click the "Execute" button
#     run_button = driver.find_element(By.ID, "execute-button")
#     run_button.click()

#     # Wait for the results table
#     WebDriverWait(driver, 10).until(
#         EC.presence_of_element_located((By.CSS_SELECTOR, "table.table"))
#     )
#     time.sleep(2)

#     # Parse results
#     table_rows = driver.find_elements(By.CSS_SELECTOR, "table.table tbody tr")
#     results = []
#     for row in table_rows:
#         cols = row.find_elements(By.TAG_NAME, "td")
#         results.append([col.text for col in cols])

#     # print("SPARQL Results:", results)
#     try:
#         wiki_id_target = results[4][0].replace('wd:','')
#         print('>>>>>', wiki_id_target)
#     except:
#         print('failed to get the exact result')

# except Exception as e:
#     print("Error:", e)
# finally:
#     pass
#     driver.quit()

In [9]:
from src import selenbeau

In [12]:
importlib.reload(selenbeau)

<module 'src.selenbeau' from 'C:\\Users\\uthop\\notebooks\\tasks\\250623_WikiLists\\src\\selenbeau.py'>

In [13]:
# import requests
# from bs4 import BeautifulSoup

# url = "https://www.wikidata.org/wiki/Q15873"

# # Fetch the HTML
# response = requests.get(url)
# soup = BeautifulSoup(response.text, "html.parser")

# # Step 1: Find all label elements
# labels = soup.find_all("div", class_="wikibase-statementgroupview-property-label")

# for label in labels:
#     if label.text.strip().lower() == "date of birth":
#         # Step 2: Get the parent container
#         parent = label.find_parent("div", class_="wikibase-statementgroupview")

#         # Step 3: Find the value inside this parent
#         value = parent.find("div", class_="wikibase-snakview-value wikibase-snakview-variation-valuesnak")
#         if value:
#             print("Date of birth:", value.text.strip())
#         else:
#             print("Date of birth value not found")
#         break


In [14]:
# selenbeau.get_dob_from_wikidata(fmid)

# Main Process

In [10]:
from src import selenbeau
import urllib.parse

In [11]:
importlib.reload(selenbeau)

<module 'src.selenbeau' from 'C:\\Users\\uthop\\notebooks\\tasks\\250623_WikiLists\\src\\selenbeau.py'>

In [12]:
def get_page_name(url):
    page_name = url.replace('https://en.wikipedia.org/wiki/','')
    page_name = page_name.replace('_',' ')
    page_name = urllib.parse.unquote(page_name)
    if ':' in page_name:
        return None
    return page_name

In [13]:
try:
    with open(load_path, 'rb') as f:
        people_dob = pickle.load(f)
    errors = list()
except:
    people_dob = dict()
    errors = list()

In [14]:
import concurrent.futures

In [15]:
def process_url(url):
    if url in people_dob:
        return None
    tracer = {'url': url}
    page_name = get_page_name(url)
    if page_name is None:
        return None
    tracer['page_name'] = page_name
    try:
        page_id = selenbeau.get_wiki_id_from_page_title(page_title=page_name, open_browser=False)
        tracer['page_id'] = page_id
        dob = selenbeau.get_dob_from_wikidata(page_id)
        tracer['dob'] = dob
        return tracer
    except Exception:
        errors.append(tracer)
        return None

In [16]:
china_people_dob_path = os.path.join('data', 'outputs', 'walk_wiki', 'china_people_dob.pickle')
processed_core_urls_path = os.path.join('data', 'outputs', 'walk_wiki', 'core_urls_processed.pickle')

In [17]:
with open(china_people_dob_path, 'rb') as f:
    people_dob = pickle.load(f)

with open(processed_core_urls_path, 'rb') as f:
    core_urls_processed = pickle.load(f)

In [18]:
for core_url in core_urls:
    inner_urls = extract_people_links(core_url)
    if core_url in core_urls_processed:
        continue
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        results = list(tqdm.tqdm(executor.map(process_url, inner_urls), total=len(inner_urls), desc=core_url))

    for result in results:
        if result:
            people_dob[result['url']] = result

    core_urls_processed.append(core_url)
    
    with open(china_people_dob_path, 'wb') as f:
        pickle.dump(people_dob, f)
        
    with open(processed_core_urls_path, 'wb') as f:
        pickle.dump(core_urls_processed, f)

https://en.wikipedia.org/wiki/List_of_Chinese_by_net_worth: 100%|████████████████████| 312/312 [41:49<00:00,  8.04s/it]


In [23]:
hah = pd.DataFrame(list(people_dob.values()))

In [37]:
hah[(hah['page_id'].str.startswith('Q')) & (hah['dob'].str.len()>4)]

,url,page_name,page_id,dob
3,https://en.wikipedia.org/wiki/Li_Di,Li Di,Q27527265,15 December 1999
4,https://en.wikipedia.org/wiki/Zhou_Chengzhou,Zhou Chengzhou,Q105721640,28 December 1982
14,https://en.wikipedia.org/wiki/Xin_Fengxia,Xin Fengxia,Q8263391,26 January 1927Gregorian
16,https://en.wikipedia.org/wiki/He_Xiangning,He Xiangning,Q8248830,27 June 1878Gregorian
24,https://en.wikipedia.org/wiki/Aowen_Jin,Aowen Jin,Q18688034,July 1978
...,...,...,...,...
1528,https://en.wikipedia.org/wiki/Zhao_Leji,Zhao Leji,Q127295,8 March 1957
1551,https://en.wikipedia.org/wiki/Wang_Xiaohong,Wang Xiaohong,Q18109475,11 July 1957
1562,https://en.wikipedia.org/wiki/Shi_Taifeng,Shi Taifeng,Q8292435,September 1956
1569,https://en.wikipedia.org/wiki/Chen_Wenqing,Chen Wenqing,Q8340313,January 1960
